<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Arabic/FASTTEXT_biGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arabic Language

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Arabic/

/content/drive/My Drive/dataset/OffenseEval2020/data/Arabic


In [3]:
ls

cc.ar.300.vec     offenseval-ar-dev-v1.tsv       readme-data-ar.txt
cc.ar.300.vec.gz  offenseval-ar-training-v1.tsv


In [0]:
import pandas as pd
import numpy as np
train=pd.read_csv( 'offenseval-ar-training-v1.tsv',sep="\t")
train=train.sample(frac=1)

In [5]:
train.head()

,id,tweet,subtask_a
4743,4815,يا رخمت يا بي يا تُقله يا ساتر URL,NOT
1437,1438,@USER انا الحمد لله برضو متفرحتش ولا فاكر الما...,NOT
6366,6509,RT @USER: #صدق_المثل يا مأمنة للمَي بالغربال ي...,NOT
5555,5653,الصمله يا عميد اسيا يا كبير المملكه,NOT
2722,2729,@USER يا حظّ المكان بيك يا حظّ من هم حواليك يا...,NOT


In [6]:
train.tail()

,id,tweet,subtask_a
5492,5590,@USER وانت طيب يا صاصا يا صاحبي وبخير دايما يا...,NOT
442,443,RT @USER: هاه، يا ولاد الكلاب يا جزم..؟! URL,OFF
570,571,RT @USER: تسلّطي يا مليكتي<LF>وتلالأي يا نجمتي...,NOT
1538,1539,@USER الحمدلله عايشين اعز عيشه في حكم ال سعود ...,OFF
5395,5493,يا بداية المحبه .. يا نهاية الوله.,NOT


In [0]:
val=pd.read_csv( 'offenseval-ar-dev-v1.tsv',sep="\t")

In [0]:
train_labels=train['subtask_a']
X_train=train['tweet']
y_train=pd.factorize(train_labels)[0]

In [9]:
import collections
collections.Counter(train_labels)

Counter({'NOT': 5468, 'OFF': 1371})

In [0]:
val_labels=val['subtask_a']
X_val=val['tweet']
y_val=pd.factorize(val_labels)[0]

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(X_train)
sequences = tokenize.texts_to_sequences(X_train)
word_index = tokenize.word_index
X_train_sequences = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [12]:
len(X_train_sequences),len(train_labels)

(6839, 6839)

In [0]:

val_sequences = tokenize.texts_to_sequences(X_val)
X_val_sequences = sequence.pad_sequences(val_sequences,maxlen=max_len)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.vec.gz
# import gzip
# import shutil
# with gzip.open('cc.ar.300.vec.gz', 'rb') as f_in:
#     with open('cc.ar.300.vec', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [17]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

10001


In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

embeddings=build_matrix('cc.ar.300.vec', word_index)


In [0]:
model = Sequential()
model.add(Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False))
model.add(Bidirectional(CuDNNGRU(150, return_sequences = True)))
#model.add(Bidirectional(CuDNNGRU(50, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(50, activation="relu"))
model.add(Dense(20, activation="relu"))

model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [22]:

model.fit(X_train_sequences,y_train,batch_size=256,epochs=10,verbose=2,class_weight=class_weights,validation_data=(X_val_sequences,y_val),callbacks=[reduce_lr])

Train on 6839 samples, validate on 1000 samples
Epoch 1/10
 - 17s - loss: 0.6174 - f1: 0.4545 - val_loss: 0.4555 - val_f1: 0.5693
Epoch 2/10
 - 2s - loss: 0.4201 - f1: 0.6385 - val_loss: 0.2942 - val_f1: 0.7014
Epoch 3/10
 - 2s - loss: 0.3507 - f1: 0.7030 - val_loss: 0.3176 - val_f1: 0.7110
Epoch 4/10
 - 2s - loss: 0.3182 - f1: 0.7272 - val_loss: 0.2918 - val_f1: 0.7328
Epoch 5/10
 - 2s - loss: 0.2930 - f1: 0.7539 - val_loss: 0.2705 - val_f1: 0.7350
Epoch 6/10
 - 2s - loss: 0.2738 - f1: 0.7592 - val_loss: 0.2933 - val_f1: 0.7179
Epoch 7/10
 - 2s - loss: 0.2527 - f1: 0.7783 - val_loss: 0.2960 - val_f1: 0.7184

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 8/10
 - 2s - loss: 0.2257 - f1: 0.8052 - val_loss: 0.2743 - val_f1: 0.7416
Epoch 9/10
 - 2s - loss: 0.2143 - f1: 0.8128 - val_loss: 0.2687 - val_f1: 0.7346
Epoch 10/10
 - 2s - loss: 0.2100 - f1: 0.8193 - val_loss: 0.2789 - val_f1: 0.7400


In [23]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_val_sequences, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

1000/1000 [==============================] - 0s 326us/step
              precision    recall  f1-score   support

           0       0.96      0.91      0.94       821
           1       0.68      0.83      0.74       179

    accuracy                           0.90      1000
   macro avg       0.82      0.87      0.84      1000
weighted avg       0.91      0.90      0.90      1000



In [0]:
import keras
keras.backend.clear_session()